## Import Libraries

In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
from arch import arch_model
from scipy.optimize import minimize

## Dynamic Portfolio: <span style="font-size:20px; color:white; font-weight:900">Risk Parity (Strategy) + GARCH (Simulation)</span>

In [13]:
import numpy as np
import pandas as pd
import yfinance as yf
from arch import arch_model
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import warnings

# Suppress warnings and messages
warnings.filterwarnings("ignore")

# Step 1: Download historical data
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']  # Example assets
data = yf.download(tickers + ['^GSPC'], start='2015-01-01', end='2025-01-01', progress=False)['Close']
returns = np.log(data / data.shift(1)).dropna()

# Step 2: Fit GARCH(1,1) model and forecast volatility
def garch_volatility_forecast(returns, horizon=1):
    if returns.isnull().all():  # Handle empty returns
        return np.nan
    try:
        model = arch_model(returns.dropna(), vol='Garch', p=1, q=1)
        res = model.fit(disp='off')
        forecast = res.forecast(start=returns.index[-1], horizon=horizon, reindex=False)
        return np.sqrt(forecast.variance.iloc[-1])  # Annualized volatility
    except:
        return np.nan  # Return NaN if fitting fails

# Step 3: Compute Risk Parity Weights
def risk_parity_weights(vols):
    vols = vols.fillna(method='bfill')  # Fill missing volatilities
    if vols.isnull().any() or (vols == 0).any().any():
        return pd.Series(np.nan, index=vols.index)  # Return NaN weights if invalid
    inv_vols = 1 / vols
    weights = inv_vols / inv_vols.sum()
    return weights

# Step 4: Portfolio Rebalancing Function
def rebalance_portfolio(returns, rebalance_freq='M'):
    weight_history = []
    dates = []
    
    for date, ret in returns.resample(rebalance_freq).mean().iterrows():
        vol_forecasts = pd.Series({ticker: garch_volatility_forecast(returns[ticker].loc[:date]) for ticker in tickers})
        new_weights = risk_parity_weights(vol_forecasts)
        if new_weights.isnull().any():  # Skip rebalancing if NaN weights
            continue
        weight_history.append(new_weights)
        dates.append(date)
    
    return pd.DataFrame(weight_history, index=dates)

# Compute dynamic weights over time
dynamic_weights = rebalance_portfolio(returns)
dynamic_weights = dynamic_weights.reindex(returns.index, method='ffill')  # Forward-fill weights

# Step 5: Backtest Performance
if dynamic_weights.isnull().all().all():
    raise ValueError("All weights are NaN. Check data and GARCH fitting.")

portfolio_returns = (returns[tickers] * dynamic_weights).sum(axis=1)
sp500_returns = returns['^GSPC']

cumulative_portfolio_returns = (1 + portfolio_returns).cumprod()
cumulative_sp500_returns = (1 + sp500_returns).cumprod()

# Step 6: Plot the Results
plt.figure(figsize=(12, 6))
plt.plot(cumulative_portfolio_returns, label='Risk Parity Portfolio', linewidth=2)
plt.plot(cumulative_sp500_returns, label='S&P 500 Index', linewidth=2, linestyle='dashed')
plt.legend()
plt.title('Backtest Performance: Risk Parity Portfolio vs S&P 500')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.grid()
plt.show()


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [11]:
cumulative_portfolio_returns

Date
2015-01-05                           1
2015-01-06                           1
2015-01-07                           1
2015-01-08                           1
2015-01-09                           1
                        ...           
2024-12-24    h.1   NaN
dtype: float64
2024-12-26    h.1   NaN
dtype: float64
2024-12-27    h.1   NaN
dtype: float64
2024-12-30    h.1   NaN
dtype: float64
2024-12-31    h.1   NaN
dtype: float64
Length: 2515, dtype: object